# Read and load results

In [1]:
from functions import *
from Classes import *

# Define the use technologies and regions(s)
region = 'ESP'  
setup = {
    f'{region}': {
        'OCGT': True,
        'CCGT': False,
        'battery storage': True,
        'onwind': True,
        'offwind': False,
        'solar': True,
        'electrolysis': True,
        'fuel cell': True,
        'Hydrogen storage': True,
        'Reservoir hydro storage': True,
        'Load shedding': False
    }
}

# Default weather, hydro and demand years
w_year_exp = 2011
h_year_exp = 2007
d_year_exp = 2018

# Dispatch and rolling horizon settings
w_year_dispatch = 2017
h_year_dispatch = 2007
d_year_dispatch = 2019

weather_years = All_data['solar'].index.year.unique()


# Comparing two networks

In [2]:
folder_exp_1 = f"N_EXP_d_{d_year_exp}_h_{h_year_exp}"
folder_exp_2 = f"N_EXP_d_{d_year_exp}_h_{h_year_exp}_hMC"

networks_exp_1, names_exp_1 = load_networks(folder_exp_1)
networks_exp_2, names_exp_2 = load_networks(folder_exp_2)


INFO:pypsa.io:Imported network N_w-1979_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1980_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1981_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1982_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1983_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1984_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1985_d-2018_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_w-1986_d-2018_h-2007_ESP.nc has buses, carriers, generato

# Build df and tables to store results

In [40]:
if len(networks_exp_1) != len(networks_exp_2):
    print("Warning: The number of loaded exp_1 and exp_2 networks do not match!")

results = []
for i, (name_exp_1, name_exp_2) in enumerate(zip(names_exp_1, names_exp_2)):
    net_exp_1 = networks_exp_1[name_exp_1]
    net_exp_2 = networks_exp_2[name_exp_2]

    exp_1_cost = (net_exp_1.buses_t['marginal_price']["electricity bus"].values *
                  net_exp_1.loads_t['p_set']['load']).sum() / 1e6
    exp_2_cost = (net_exp_2.buses_t['marginal_price']["electricity bus"].values *
                  net_exp_2.loads_t['p_set']['load']).sum() / 1e6
    diff = exp_2_cost - exp_1_cost

    ls_exp_1 = ((net_exp_1.generators_t.p["Load shedding"] *
                 net_exp_1.generators.at["Load shedding", "marginal_cost"]).sum() / 1e6
                if "Load shedding" in net_exp_1.generators.index else 0.0)

    ls_exp_2 = ((net_exp_2.generators_t.p["Load shedding"] *
                 net_exp_2.generators.at["Load shedding", "marginal_cost"]).sum() / 1e6
                if "Load shedding" in net_exp_2.generators.index else 0.0)

    obj_exp_1 = round(net_exp_1.objective/1e6,2)
    obj_exp_2 = round(net_exp_2.objective/1e6,2)

    results.append({
        'pair_idx': i,
        'exp_1_key': name_exp_1,
        'exp_2_key': name_exp_2,
        'exp_1_objective [mEUR]': obj_exp_1,
        'exp_2_objective [mEUR]': obj_exp_2,
        'Objective_Difference [mEUR]': round(obj_exp_2 - obj_exp_1, 1),
        'exp_1_cost [mEUR]': round(exp_1_cost, 1),
        'exp_2_cost [mEUR]': round(exp_2_cost, 1),
        'Cost_Difference [mEUR]': round(diff, 1),
        'exp_1_Load_shedding_cost [mEUR]': round(ls_exp_1, 1),
        'exp_2_Load_shedding_cost [mEUR]': round(ls_exp_2, 1)
        
    })

results_df = pd.DataFrame(results)
results_df

,pair_idx,exp_1_key,exp_2_key,exp_1_objective [mEUR],exp_2_objective [mEUR],Objective_Difference [mEUR],exp_1_cost [mEUR],exp_2_cost [mEUR],Cost_Difference [mEUR],exp_1_Load_shedding_cost [mEUR],exp_2_Load_shedding_cost [mEUR]
0,0,N_w-1979_d-2018_h-2007_ESP,N_w-1979_d-2018_h-2007_ESP_hMC,12887.35,12930.67,43.3,15072.5,15072.5,0.0,0.0,0.0
1,1,N_w-1980_d-2018_h-2007_ESP,N_w-1980_d-2018_h-2007_ESP_hMC,13131.75,13175.07,43.3,15317.0,15317.0,0.0,0.0,0.0
2,2,N_w-1981_d-2018_h-2007_ESP,N_w-1981_d-2018_h-2007_ESP_hMC,13126.68,13169.97,43.3,15305.1,15305.1,0.0,0.0,0.0
3,3,N_w-1982_d-2018_h-2007_ESP,N_w-1982_d-2018_h-2007_ESP_hMC,13348.45,13391.77,43.3,15533.7,15533.7,0.0,0.0,0.0
4,4,N_w-1983_d-2018_h-2007_ESP,N_w-1983_d-2018_h-2007_ESP_hMC,13648.77,13692.10,43.3,15834.0,15834.0,0.0,0.0,0.0
5,5,N_w-1984_d-2018_h-2007_ESP,N_w-1984_d-2018_h-2007_ESP_hMC,12717.22,12760.53,43.3,14898.1,14898.2,0.0,0.0,0.0
6,6,N_w-1985_d-2018_h-2007_ESP,N_w-1985_d-2018_h-2007_ESP_hMC,13294.94,13338.25,43.3,15478.8,15478.8,0.0,0.0,0.0
7,7,N_w-1986_d-2018_h-2007_ESP,N_w-1986_d-2018_h-2007_ESP_hMC,13058.31,13101.63,43.3,15243.6,15243.6,0.0,0.0,0.0
8,8,N_w-1987_d-2018_h-2007_ESP,N_w-1987_d-2018_h-2007_ESP_hMC,13312.79,13356.11,43.3,15497.4,15497.4,0.0,0.0,0.0
9,9,N_w-1988_d-2018_h-2007_ESP,N_w-1988_d-2018_h-2007_ESP_hMC,13160.19,13203.51,43.3,15345.4,15345.4,0.0,0.0,0.0


In [35]:
results_df

,pair_idx,exp_1_key,exp_2_key,exp_1_cost_MEUR,exp_2_cost_MEUR,Difference_MEUR,exp_1_Load_shedding_cost_MEUR,exp_2_Load_shedding_cost_MEUR,exp_1_objective,exp_2_objective
0,0,N_w-1979_d-2018_h-2007_ESP,N_w-1979_d-2018_h-2007_ESP_hMC,15072.5,15072.5,0.0,0.0,0.0,12.887348,12.930670
1,1,N_w-1980_d-2018_h-2007_ESP,N_w-1980_d-2018_h-2007_ESP_hMC,15317.0,15317.0,0.0,0.0,0.0,13.131749,13.175071
2,2,N_w-1981_d-2018_h-2007_ESP,N_w-1981_d-2018_h-2007_ESP_hMC,15305.1,15305.1,0.0,0.0,0.0,13.126677,13.169974
3,3,N_w-1982_d-2018_h-2007_ESP,N_w-1982_d-2018_h-2007_ESP_hMC,15533.7,15533.7,0.0,0.0,0.0,13.348453,13.391775
4,4,N_w-1983_d-2018_h-2007_ESP,N_w-1983_d-2018_h-2007_ESP_hMC,15834.0,15834.0,0.0,0.0,0.0,13.648774,13.692096
5,5,N_w-1984_d-2018_h-2007_ESP,N_w-1984_d-2018_h-2007_ESP_hMC,14898.1,14898.2,0.0,0.0,0.0,12.717218,12.760525
6,6,N_w-1985_d-2018_h-2007_ESP,N_w-1985_d-2018_h-2007_ESP_hMC,15478.8,15478.8,0.0,0.0,0.0,13.294936,13.338251
7,7,N_w-1986_d-2018_h-2007_ESP,N_w-1986_d-2018_h-2007_ESP_hMC,15243.6,15243.6,0.0,0.0,0.0,13.058306,13.101628
8,8,N_w-1987_d-2018_h-2007_ESP,N_w-1987_d-2018_h-2007_ESP_hMC,15497.4,15497.4,0.0,0.0,0.0,13.312791,13.356110
9,9,N_w-1988_d-2018_h-2007_ESP,N_w-1988_d-2018_h-2007_ESP_hMC,15345.4,15345.4,0.0,0.0,0.0,13.160190,13.203513


## Functions to calc results

In [29]:
def calculate_cost_recovery_with_storage(network_exp_1, network_exp_2):
    results = []

    for model_label, network in zip(["exp_1", "exp_2"], [network_exp_1, network_exp_2]):

        # Generators
        for gen in network.generators.index:
            carrier = network.generators.at[gen, "carrier"]
            bus = network.generators.at[gen, "bus"]
            p_nom = network.generators.at[gen, "p_nom"]
            marginal_cost = network.generators.at[gen, "marginal_cost"]
            capital_cost = 0 if gen == "load_shedding" else network.generators.at[gen, "capital_cost"]

            dispatch = network.generators_t.p[gen]
            prices = network.buses_t.marginal_price[bus]

            revenue = (dispatch * prices).sum()
            production_cost = (dispatch * marginal_cost).sum()
            capex = p_nom * capital_cost
            profit = revenue - (capex + production_cost)

            results.append({
                "Model": model_label,
                "name": gen,
                "carrier": carrier,
                "revenue [MEUR]": round(revenue / 1e6, 1),
                "production cost [MEUR]": round(production_cost / 1e6, 1),
                "capital cost [MEUR]": round(capex / 1e6, 1),
                "total cost [MEUR]": round((production_cost + capex) / 1e6, 1),
                "profit [MEUR]": round(profit / 1e6, 1)
            })

        # Storage Units
        for su in network.storage_units.index:
            carrier = network.storage_units.at[su, "carrier"]
            bus = network.storage_units.at[su, "bus"]
            p_nom = network.storage_units.at[su, "p_nom"]
            capital_cost = network.storage_units.at[su, "capital_cost"]

            dispatch = network.storage_units_t.p_dispatch[su]
            prices = network.buses_t.marginal_price[bus]

            revenue = (dispatch * prices).sum()
            marginal_cost = network.storage_units.at[su, "marginal_cost"]
            production_cost = (dispatch * marginal_cost).sum()
            capex = p_nom * capital_cost
            profit = revenue - (capex + production_cost)

            results.append({
                "Model": model_label,
                "name": su,
                "carrier": carrier,
                "revenue [MEUR]": round(revenue / 1e6, 1),
                "production cost [MEUR]": round(production_cost / 1e6, 1),
                "capital cost [MEUR]": round(capex / 1e6, 1),
                "total cost [MEUR]": round((production_cost + capex) / 1e6, 1),
                "profit [MEUR]": round(profit / 1e6, 1)
            })

    return pd.DataFrame(results)


In [31]:
CR_1995_hydro = calculate_cost_recovery_with_storage(
    networks_exp_1[names_exp_1[16]],
    networks_exp_2[names_exp_2[16]]
)
CR_1995_hydro

,Model,name,carrier,revenue [MEUR],production cost [MEUR],capital cost [MEUR],total cost [MEUR],profit [MEUR]
0,exp_1,OCGT,gas,4579.0,3576.1,0.0,3576.1,1002.9
1,exp_1,onwind,onwind,7083.6,230.0,0.0,230.0,6853.6
2,exp_1,solar,solar,1004.4,0.2,0.0,0.2,1004.1
3,exp_1,Reservoir hydro storage,hydro,2170.4,0.0,0.0,0.0,2170.4
4,exp_2,OCGT,gas,4579.0,3576.1,0.0,3576.1,1002.9
5,exp_2,onwind,onwind,7083.8,230.2,0.0,230.2,6853.6
6,exp_2,solar,solar,1004.4,0.2,0.0,0.2,1004.1
7,exp_2,Reservoir hydro storage,hydro,2170.2,43.1,0.0,43.1,2127.1


## Look at results

# Plot results

## Result type 1

# TEST MC on hydro

In [8]:
class Build_network_capacity_exp:
    def __init__(
        self,
        weather_year: int = 2011,
        hydro_year: int = 2011,
        demand_year: int = 2018,
        data: dict = None,
        cost_data: tuple = None,
        setup: dict = None
    ):
        if setup is None:
            setup = {
                'NOR': {
                    'OCGT': True,
                    'CCGT': False,
                    'battery storage': True,
                    'onwind': True,
                    'offwind': False,
                    'solar': True,
                    'electrolysis': True,
                    'fuel cell': True,
                    'Hydrogen storage': True,
                    'Reservoir hydro storage': True,
                    'load shedding': True
                }
            }

        self.weather_year = weather_year
        self.hydro_year = hydro_year
        self.demand_year = demand_year
        self.setup = setup
        self.region = list(setup.keys())[0]  # Single region expected

        self.costs = cost_data.costs
        self.cost_units = cost_data.units

        self.all_data = data if data is not None else load_all_data()
        self.data_dict = {self.region: self.extract_data(self.region, self.weather_year, self.hydro_year, self.demand_year)}
        
        self.network = pypsa.Network()
        self.hours_in_year = pd.date_range(f'{weather_year}-01-01 00:00', f'{weather_year}-12-31 23:00', freq='h')
        if len(self.hours_in_year) > 8760:
            self.hours_in_year = self.hours_in_year[self.hours_in_year.strftime('%m-%d') != '02-29']
        self.network.set_snapshots(self.hours_in_year.values)

        self.carriers = [
            'gas', 'onwind', 'offwind', 'solar',
            'battery charge', 'battery discharge', 'hydro',
            'electrolysis', 'fuel cell', 'hydrogen', 'load shedding'
        ]

        self.colors = {
            'gas': 'gray', 'onwind': 'lightblue', 'offwind': 'dodgerblue', 'load shedding': 'red',
            'solar': 'orange', 'battery charge': 'gold', 'battery discharge': 'darkorange',
            'electrolysis': 'green', 'fuel cell': 'limegreen', 'hydrogen': 'deepskyblue', 'hydro': 'slateblue'
        }

        self.network.add("Carrier",
            self.carriers,
            color=[self.colors[c] for c in self.carriers],
            co2_emissions=[self.costs.at[c, "CO2 intensity"] if c in self.costs.index else 0.0 for c in self.carriers]
        )


        self.network.add("Bus", 'electricity bus')
        self.network.add("Bus", 'hydrogen bus')
        self.network.add("Load", 'load',
                         bus='electricity bus',
                         p_set=self.data_dict[self.region]['demand'].values.flatten()
                         )


        technologies = self.setup[self.region].keys()
        for tech in technologies:
            if not self.setup[self.region][tech]:
                continue

            if tech in ['OCGT', 'CCGT']:
                self.network.add("Generator", tech,
                    bus='electricity bus',
                    p_nom_extendable=True,
                    carrier='gas',
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    marginal_cost=self.costs.at[tech, "marginal_cost"]
                    )

            elif tech == 'load shedding':
                self.network.add("Generator", tech,
                    bus="electricity bus",
                    p_nom_extendable=True,
                    marginal_cost=2000,   # €/MWh, can adjust based on VoLL
                    capital_cost=0,
                    carrier="load shedding"
                    )
                
            elif tech == 'solar':
                self.network.add("Generator", tech,
                    bus='electricity bus',
                    p_nom_extendable=True,
                    carrier='solar',
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    marginal_cost=self.costs.at[tech, "marginal_cost"],
                    p_max_pu=self.data_dict[self.region]['solar'].values.flatten()
                    )

            elif tech == 'onwind':
                self.network.add("Generator", tech,
                    bus='electricity bus',
                    p_nom_extendable=True,
                    carrier='onwind',
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    marginal_cost=self.costs.at[tech, "marginal_cost"],
                    p_max_pu=self.data_dict[self.region]['onwind'].values.flatten()
                    )

            elif tech == 'offwind':
                self.network.add("Generator", tech,
                    bus='electricity bus',
                    p_nom_extendable=True,
                    carrier='offwind',
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    marginal_cost=self.costs.at[tech, "marginal_cost"],
                    p_max_pu=self.data_dict[self.region]['offwind'].values.flatten()
                    )

            elif tech == 'battery storage':
                self.network.add("Bus", 'battery bus')

                self.network.add("Link", 'battery charge',
                    bus0='electricity bus',
                    bus1='battery bus',
                    carrier='battery charge',
                    p_nom_extendable=True,
                    capital_cost=self.costs.at["battery inverter", "capital_cost"]/2,    # Divide by two as only one inverter will be baught in reality
                    efficiency=self.costs.at["battery inverter", "efficiency"]
                    )

                self.network.add("Link", 'battery discharge',
                    bus0='battery bus',
                    bus1='electricity bus',
                    carrier='battery discharge',
                    p_nom_extendable=True,
                    capital_cost=self.costs.at["battery inverter", "capital_cost"]/2,     # Divide by two as only one inverter will be baught in reality
                    efficiency=self.costs.at["battery inverter", "efficiency"]
                    )        

                self.network.add("Store", tech,
                    bus='battery bus',
                    e_nom_extendable=True,
                    e_cyclic=False,
                    capital_cost=self.costs.at[tech, "capital_cost"]
                    )

            elif tech == 'electrolysis':
                self.network.add("Link", tech,
                    bus0='electricity bus',
                    bus1='hydrogen bus',
                    carrier='electrolysis',
                    p_nom_extendable=True,
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    efficiency=self.costs.at[tech, "efficiency"]
                    )

            elif tech == 'fuel cell':
                self.network.add("Link", tech,
                    bus0='hydrogen bus',
                    bus1='electricity bus',
                    carrier='fuel cell',
                    p_nom_extendable=True,
                    capital_cost=self.costs.at[tech, "capital_cost"],
                    efficiency=self.costs.at[tech, "efficiency"]
                    )

            elif tech == 'Hydrogen storage':
                self.network.add("Store", tech,
                    bus='hydrogen bus',
                    e_nom_extendable=True,
                    e_cyclic=False,
                    capital_cost=self.costs.at["H2 (l) storage tank", "capital_cost"],
                    carrier='hydrogen storage'
                    )
                
            elif tech == 'Reservoir hydro storage':
                self.network.add("StorageUnit", tech,
                    bus='electricity bus',
                    carrier='hydro',
                    p_nom_extendable=False,
                    p_nom = 12700,  # 12 GW
                    max_hours=1300,
                    efficiency_store=0,
                    efficiency_dispatch=self.costs.at["Pumped-Storage-Hydro-bicharger", "efficiency"],
                    cyclic_state_of_charge=False,
                    state_of_charge_initial= (12700 * 1300)*0.3 ,  # Initial storage capacity in MWh
                    inflow=self.data_dict[self.region]['hydro'].values.flatten(),
                    marginal_cost=self.costs.at["onwind", "marginal_cost"]*1.2,  # higher than wind to prioritize wind usage
                    capital_cost=0
                    )

    def extract_data(self, region: str, weather_year: int, hydro_year: int, demand_year: int):
        extracted = {}
        if demand_year not in self.all_data["demand"].index.year:
            print(f"Demand year {demand_year} not in data. Using 2017 instead.")
            demand_year = 2017
        if weather_year not in self.all_data["solar"].index.year:
            print(f"Weather year {weather_year} not in data. Using 2012 instead.")
            weather_year = 2012
        if hydro_year not in self.all_data["hydro_inflow"].index.year:
            print(f"Hydro year {hydro_year} not in data. Using 2011 instead.")
            hydro_year = 2011

        if region in self.all_data["demand"].columns:
            demand_series = self.all_data["demand"].loc[self.all_data["demand"].index.year == demand_year, region]
            extracted["demand"] = demand_series[demand_series.index.strftime('%m-%d') != '02-29'][:8760]

        for carrier in ["solar", "onwind", "offwind"]:
            if region in self.all_data[carrier].columns:
                weather_series = self.all_data[carrier].loc[self.all_data[carrier].index.year == weather_year, region]
                extracted[carrier] = weather_series[weather_series.index.strftime('%m-%d') != '02-29'][:8760]

        if region in self.all_data["hydro_inflow"].columns:
            hydro_series = self.all_data["hydro_inflow"].loc[self.all_data["hydro_inflow"].index.year == hydro_year, region]
            extracted["hydro"] = hydro_series[hydro_series.index.strftime('%m-%d') != '02-29'][:8760]

        return extracted  

In [12]:
N_class = Build_network_capacity_exp(weather_year=1995, hydro_year=h_year_exp, demand_year=d_year_exp,
    data=All_data, cost_data=Cost, setup=setup_exp)
N = N_class.network

silent_optimize(N)

print_Results(N)
print("Hydro marginal cost:", N.storage_units.at['Reservoir hydro storage', 'marginal_cost'])


Writing continuous variables.: 100%|██████████| 11/11 [00:00<00:00, 39.49it/s]



Objective value (MEUR): 12797

Installed generator capacities (MW):
Generator
OCGT      20577.0
onwind    63789.0
solar     15909.0

Installed store energy capacities (MWh):
Store
battery storage     2918.0
Hydrogen storage       0.0

Installed hydro power capacity (MW):
StorageUnit
Reservoir hydro storage    12700.0

Installed link power capacities (MW):
Link
battery charge        509.0
battery discharge    1567.0
electrolysis            0.0
fuel cell               0.0
Hydro marginal cost: 1.8095999999999999


In [13]:
obj_exp_1 = networks_exp_1[names_exp_1[16]].objective
obj_exp_2 = networks_exp_2[names_exp_2[16]].objective

print("Objective value (exp_1):", obj_exp_1)
print("Objective value (exp_2):", obj_exp_2)

Objective value (exp_1): 12753319434.89959
Objective value (exp_2): 12796602797.481852


In [10]:
region = 'ESP'  
setup_exp = {
    f'{region}': {
        'OCGT': True,
        'CCGT': False,
        'battery storage': True,
        'onwind': True,
        'offwind': False,
        'solar': True,
        'electrolysis': True,
        'fuel cell': True,
        'Hydrogen storage': True,
        'Reservoir hydro storage': True,
        'Load shedding': False
    }
}

# Test COST values

In [20]:
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{2025}.csv"
df = pd.read_csv(url, index_col=[0, 1])
df.head()

value           unit  \
technology                       parameter                                    
Alkaline electrolyzer large size FOM                  4.6000         %/year   
                                 VOM                  0.2389     EUR/MWh_H2   
                                 electricity-input    1.3800  MWh_el/MWh_H2   
                                 investment         429.0306         EUR/kW   
                                 lifetime            40.0000          years   

                                                                                               source  \
technology                       parameter                                                              
Alkaline electrolyzer large size FOM                JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 VOM                JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 electricity-input  JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 investment         JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 lifetime           JRC, 01_JRC-EU-TIMES Full model, https://zenod...   

                                                         further description  \
technology                       parameter                                     
Alkaline electrolyzer large size FOM                Reference capacity 72 MW   
                                 VOM                Reference capacity 72 MW   
                                 electricity-input  Reference capacity 72 MW   
                                 investment         Reference capacity 72 MW   
                                 lifetime           Reference capacity 72 MW   

                                                    currency_year  
technology                       parameter                         
Alkaline electrolyzer large size FOM                       2010.0  
                                 VOM                       2010.0  
                                 electricity-input            NaN  
                                 investment                2010.0  
                                 lifetime                     NaN

In [24]:
investment_units = df[df.index.get_level_values(1).str.contains("investment")]["unit"].unique().tolist()
investment_units

tech_units = {}
for tech in setup_exp[region]:
    # Find all units for this technology in the cost dataframe, only for parameter 'investment'
    units = df.loc[(df.index.get_level_values(0) == tech) & (df.index.get_level_values(1) == "investment"), "unit"].unique().tolist()
    tech_units[tech] = units

tech_units

{'OCGT': ['EUR/kW'],
 'CCGT': ['EUR/kW'],
 'battery storage': ['EUR/kWh'],
 'onwind': ['EUR/kW'],
 'offwind': ['EUR/kW_e, 2020'],
 'solar': ['EUR/kW_e'],
 'electrolysis': ['EUR/kW_e'],
 'fuel cell': ['EUR/kW_e'],
 'Hydrogen storage': [],
 'Reservoir hydro storage': [],
 'Load shedding': []}

In [49]:
for tech in setup_exp[region]:
    print(f"Investment cost for '{tech}':")
    if tech in tech_units and tech_units[tech]:
        for unit in tech_units[tech]:
            value = df.loc[(tech, "investment"), "value"] if (tech, "investment") in df.index else None
            print(f"  Unit: {unit}, Value: {value}")
    else:
        print("  No investment cost unit found.")

    # Include lifetime parameter
    lifetime = df.loc[(tech, "lifetime"), "value"] if (tech, "lifetime") in df.index else None
    print(f"  Lifetime: {lifetime}")

    # Include FOM parameter
    fom = df.loc[(tech, "FOM"), "value"] if (tech, "FOM") in df.index else None
    print(f"  FOM: {fom}")

Investment cost for 'OCGT':
  Unit: EUR/kW, Value: 470.4853
  Lifetime: 25.0
  FOM: 1.7784
Investment cost for 'CCGT':
  Unit: EUR/kW, Value: 904.7795
  Lifetime: 25.0
  FOM: 3.3392
Investment cost for 'battery storage':
  Unit: EUR/kWh, Value: 198.8558
  Lifetime: 22.5
  FOM: None
Investment cost for 'onwind':
  Unit: EUR/kW, Value: 1139.8826
  Lifetime: 28.5
  FOM: 1.2347
Investment cost for 'offwind':
  Unit: EUR/kW_e, 2020, Value: 1769.1171
  Lifetime: 30.0
  FOM: 2.3741
Investment cost for 'solar':
  Unit: EUR/kW_e, Value: 676.5703
  Lifetime: 37.5
  FOM: 1.7275
Investment cost for 'electrolysis':
  Unit: EUR/kW_e, Value: 1800.0
  Lifetime: 25.0
  FOM: 4.0
Investment cost for 'fuel cell':
  Unit: EUR/kW_e, Value: 1269.866
  Lifetime: 10.0
  FOM: 5.0
Investment cost for 'Hydrogen storage':
  No investment cost unit found.
  Lifetime: None
  FOM: None
Investment cost for 'Reservoir hydro storage':
  No investment cost unit found.
  Lifetime: None
  FOM: None
Investment cost for 'Loa

In [47]:
for tech, active in setup[region].items():
    if active and tech in Cost.costs.index:
        cap_cost = Cost.costs.loc[tech, "capital_cost"]
        unit = Cost.units.get((tech, "investment"), "")
        print(f"{tech}: {cap_cost:.2f} EUR/MW | Unit: {unit}")

# Also look up battery inverter and battery storage
if "battery inverter" in Cost.costs.index:
    cap_cost_battery_inv = Cost.costs.loc["battery inverter", "capital_cost"]
    unit_battery_inv = Cost.units.get(("battery inverter", "investment"), "")
    print(f"battery inverter: {cap_cost_battery_inv:.2f} EUR/MW | Unit: {unit_battery_inv}")



OCGT: 48739.70 EUR/MW | Unit: EUR/MW
battery storage: 17805.06 EUR/MW | Unit: EUR/MWh
onwind: 107441.46 EUR/MW | Unit: EUR/MW
solar: 63114.92 EUR/MW | Unit: EUR/MW_e
electrolysis: 226458.93 EUR/MW | Unit: EUR/MW_e
fuel cell: 244293.65 EUR/MW | Unit: EUR/MW_e
battery inverter: 33126.23 EUR/MW | Unit: EUR/MW


In [53]:
(1139.8826*1e3)-107441.46

1032441.1399999999

In [55]:
1-((1139.8826*1e3)-107441.46)/(1139.8826*1e3)

0.09425660151317339